In [7]:
import pandas as pd
import matplotlib as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.models import Sequential
from keras.layers import Conv1D, GlobalMaxPooling1D
from keras import metrics

C:\Users\Hiep Nguyen\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [44]:
data=pd.read_csv('toxic comments.csv').fillna('')
data=data.drop(['id'],1)

x_train,x_test,y_train,y_test=train_test_split(data['comment_text'],data.loc[:,data.columns != 'comment_text'],
                                                   test_size=0.3, random_state = 5)

In [21]:
data.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [45]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(list(x_train))
x_train = tok.texts_to_sequences(x_train)
x_test = tok.texts_to_sequences(x_test)
x_train = pad_sequences(x_train, maxlen=max_len)
x_test = pad_sequences(x_test, maxlen=max_len)

In [57]:
def lstm_model():
    d=0.5
    model=Sequential()
    model.add(Embedding(max_words,128))
    model.add(LSTM(128,return_sequences=True,name='lstm_layer'))
    model.add(GlobalMaxPool1D())
    model.add(Dropout(d))
    model.add(Dense(50,activation='relu'))
    model.add(Dropout(d))
    model.add(Dense(6,activation='sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [64]:
model=lstm_model()
model.fit(x_train,y_train, batch_size=32, epochs=2, validation_data=(x_test,y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, None, 128)         128000    
_________________________________________________________________
lstm_layer (LSTM)            (None, None, 128)         131584    
_________________________________________________________________
global_max_pooling1d_17 (Glo (None, 128)               0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 50)                6450      
_________________________________________________________________
dropout_34 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_34 (Dense)             (None, 6)                 306       
Total para

In [61]:
def cnn_model():
    d=0.5
    model=Sequential()
    model.add(Embedding(max_words,128))
    model.add(Dropout(d))
    model.add(Conv1D(64,
                     3,
                     padding='valid',
                     activation='relu',
                     strides=1))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(256))
    model.add(Dropout(d))
    model.add(Activation('relu'))
    model.add(Dense(6))
    model.add(Activation('sigmoid'))
    model.summary()
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc',metrics.binary_accuracy])
    return model

In [63]:
model=cnn_model()
model.fit(x_train,y_train, batch_size=32, epochs=2, validation_data=(x_test,y_test))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, None, 128)         128000    
_________________________________________________________________
dropout_31 (Dropout)         (None, None, 128)         0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, None, 64)          24640     
_________________________________________________________________
global_max_pooling1d_16 (Glo (None, 64)                0         
_________________________________________________________________
dense_31 (Dense)             (None, 256)               16640     
_________________________________________________________________
dropout_32 (Dropout)         (None, 256)               0         
_________________________________________________________________
activation_19 (Activation)   (None, 256)               0         
__________